In [2]:
import os
import torch
import torch.nn as nn
import torch.utils.data as dataf
import scipy.io as io
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
from sklearn import manifold, datasets
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import squareform
import torch.nn as nn
from sklearn.manifold import TSNE
import auxil
from torchsummary import summary

In [3]:
class HetConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,padding = None, bias = None,p = 64, g = 64):
        super(HetConv, self).__init__()
        # Groupwise Convolution
        self.gwc = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,groups=g,padding = kernel_size//3, stride = stride)
        # Pointwise Convolution
        self.pwc = nn.Conv2d(in_channels, out_channels, kernel_size=1,groups=p, stride = stride)
#         self.pwc = eSEModule(in_channels,bias=True,stride = 1 )
    def forward(self, x):
        return self.gwc(x) + self.pwc(x)



class Attention(nn.Module):
    """Attention mechanism.

    Parameters
    ----------
    dim : int
        The input and out dimension of per token features.

    n_heads : int
        Number of attention heads.

    qkv_bias : bool
        If True then we include bias to the query, key and value projections.

    attn_p : float
        Dropout probability applied to the query, key and value tensors.

    proj_p : float
        Dropout probability applied to the output tensor.


    Attributes
    ----------
    scale : float
        Normalizing consant for the dot product.

    qkv : nn.Linear
        Linear projection for the query, key and value.

    proj : nn.Linear
        Linear mapping that takes in the concatenated output of all attention
        heads and maps it into a new space.

    attn_drop, proj_drop : nn.Dropout
        Dropout layers.
    """
    def __init__(self, dim, n_heads=12, qkv_bias=True, attn_p=0., proj_p=0.):
        super().__init__()
        self.n_heads = n_heads
        self.dim = dim
        self.head_dim = dim // n_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_p)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_p)

    def forward(self, x):
        """Run forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Shape `(n_samples, n_patches + 1, dim)`.

        Returns
        -------
        torch.Tensor
            Shape `(n_samples, n_patches + 1, dim)`.
        """
        n_samples, n_tokens, dim = x.shape

        if dim != self.dim:
            raise ValueError

        qkv = self.qkv(x)  # (samples, n_patches, 3 * dim)
        qkv = qkv.reshape(
                n_samples, n_tokens, 3, self.n_heads, self.head_dim
        )  # (samples, n_patches, 3, n_heads, head_dim)
        qkv = qkv.permute(
                2, 0, 3, 1, 4
        )  # (3, n_samples, n_heads, n_patches, head_dim)

        q, k, v = qkv[0], qkv[1], qkv[2]
        k_t = k.transpose(-2, -1)  # (n_samples, n_heads, head_dim, n_patches)
        dp = (
           q @ k_t
        ) * self.scale # (n_samples, n_heads, n_patches, n_patches)
        attn = dp.softmax(dim=-1)  # (n_samples, n_heads, n_patches, n_patches)
        attn = self.attn_drop(attn)

        weighted_avg = attn @ v  # (n_samples, n_heads, n_patches, head_dim)
        weighted_avg = weighted_avg.transpose(
                1, 2
        )  # (n_samples, n_patches + 1, n_heads, head_dim)
        weighted_avg = weighted_avg.flatten(2)  # (n_samples, n_patches, dim)

        x = self.proj(weighted_avg)  # (n_samples, n_patches, dim)
        x = self.proj_drop(x)  # (n_samples, n_patches, dim)

        return x


class MLP(nn.Module):
    """Multilayer perceptron.

    Parameters
    ----------
    in_features : int
        Number of input features.

    hidden_features : int
        Number of nodes in the hidden layer.

    out_features : int
        Number of output features.

    p : float
        Dropout probability.

    Attributes
    ----------
    fc : nn.Linear
        The First linear layer.

    act : nn.GELU
        GELU activation function.

    fc2 : nn.Linear
        The second linear layer.

    drop : nn.Dropout
        Dropout layer.
    """
    def __init__(self, in_features, hidden_features, out_features, p=0.):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p)

    def forward(self, x):
        """Run forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Shape `(n_samples, n_patches + 1, in_features)`.

        Returns
        -------
        torch.Tensor
            Shape `(n_samples, n_patches +1, out_features)`
        """
        x = self.fc1(
                x
        ) # (n_samples, n_patches, hidden_features)
        x = self.act(x)  # (n_samples, n_patches, hidden_features)
        x = self.drop(x)  # (n_samples, n_patches, hidden_features)
        x = self.fc2(x)  # (n_samples, n_patches, out_features)
        x = self.drop(x)  # (n_samples, n_patches, out_features)

        return x


class Block(nn.Module):
    """Transformer block.

    Parameters
    ----------
    dim : int
        Embeddinig dimension.

    n_heads : int
        Number of attention heads.

    mlp_ratio : float
        Determines the hidden dimension size of the `MLP` module with respect
        to `dim`.

    qkv_bias : bool
        If True then we include bias to the query, key and value projections.

    p, attn_p : float
        Dropout probability.

    Attributes
    ----------
    norm1, norm2 : LayerNorm
        Layer normalization.

    attn : Attention
        Attention module.

    mlp : MLP
        MLP module.
    """
    def __init__(self, dim, n_heads, mlp_ratio=4.0, qkv_bias=True, p=0., attn_p=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim, eps=1e-6)
        self.attn = Attention(
                dim,
                n_heads=n_heads,
                qkv_bias=qkv_bias,
                attn_p=attn_p,
                proj_p=p
        )
        self.norm2 = nn.LayerNorm(dim, eps=1e-6)
        hidden_features = int(dim * mlp_ratio)
        self.mlp = MLP(
                in_features=dim,
                hidden_features=hidden_features,
                out_features=dim,
        )

    def forward(self, x):
        """Run forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Shape `(n_samples, n_patches + 1, dim)`.

        Returns
        -------
        torch.Tensor
            Shape `(n_samples, n_patches + 1, dim)`.
        """
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))

        return x


class VisionTransformer(nn.Module):
    """Simplified implementation of the Vision transformer.

    Parameters
    ----------

    in_chans : int
        Number of input channels.

    n_classes : int
        Number of classes.

    embed_dim : int
        Dimensionality of the token/patch embeddings.

    depth : int
        Number of blocks.

    n_heads : int
        Number of attention heads.

    mlp_ratio : float
        Determines the hidden dimension of the `MLP` module.

    qkv_bias : bool
        If True then we include bias to the query, key and value projections.

    p, attn_p : float
        Dropout probability.

    Attributes
    ----------
    patch_embed : PatchEmbed
        Instance of `PatchEmbed` layer.

    cls_token : nn.Parameter
        Learnable parameter that will represent the first token in the sequence.
        It has `embed_dim` elements.

    pos_emb : nn.Parameter
        Positional embedding of the cls token + all the patches.
        It has `(n_patches + 1) * embed_dim` elements.

    pos_drop : nn.Dropout
        Dropout layer.

    blocks : nn.ModuleList
        List of `Block` modules.

    norm : nn.LayerNorm
        Layer normalization.
    """
    def __init__(
            self,
            in_chans=3,
            n_classes=1000,
            embed_dim=768,
            depth=8,
            n_heads=12,
            mlp_ratio=4.,
            qkv_bias=True,
            p=0.,
            attn_p=0.,
    ):
        super().__init__()

        self.conv5 = nn.Sequential(
            nn.Conv3d(1, 8, (9, 3, 3), padding=(0,1,1), stride = 1),
            nn.BatchNorm3d(8),
            nn.ReLU()
        )
        self.conv6 = nn.Sequential(
            HetConv(8 * (in_chans - 8), 64, p=1, g = 8),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
                torch.zeros(1, 121, embed_dim)
        )
        self.pos_drop = nn.Dropout(p=p)
        self.blocks = nn.ModuleList(
            [
                Block(
                    dim=embed_dim,
                    n_heads=n_heads,
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    p=p,
                    attn_p=attn_p,
                )
                for _ in range(depth)
            ]
        )
        self.convBlock = nn.Conv3d(64, 64, (2, 3, 3), padding=(0,1,1), stride = 1)
        self.norm = nn.LayerNorm(embed_dim, eps=1e-6)
        self.pool = nn.AvgPool1d(121)
        self.head = nn.Linear(embed_dim, n_classes)


    def forward(self, x):
        """Run the forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Shape `(n_samples, in_chans, img_size, img_size)`.

        Returns
        -------
        logits : torch.Tensor
            Logits over all the classes - `(n_samples, n_classes)`.
        """
        n_samples = x.shape[0]
        x = x.unsqueeze(1)
        x = self.conv5(x)
        x = x.reshape(x.shape[0],-1,11,11)
        x = self.conv6(x) # (samples, channels, h, w)
        x = x.reshape(x.shape[0], x.shape[1], -1) # (samples, embedDim, patches)
        x = x.transpose(1,2) # (samples, patches, embedDim)
        x = x + self.pos_embed  # (n_samples, n_patches, embed_dim)
        x = self.pos_drop(x)

        for block in self.blocks:
            temp = x.transpose(1, 2) # (samples, embedDim, patches)
            xNew = block(x) # (samples, patches, embedDim)
            xNew = xNew.transpose(1,2) # (samples, embedDim, patches)
            temp = temp.unsqueeze(2) # (samples, embedDim, 1, patches)
            temp = temp.reshape(temp.shape[0], temp.shape[1], 1, 11, 11) # (samples, embedDim, 1, 11, 11)
            xNew = xNew.unsqueeze(2) # (samples, embedDim, 1, patches)
            xNew = xNew.reshape(xNew.shape[0], xNew.shape[1], 1, 11, 11) # (samples, embedDim, 1, 11, 11)
            x = torch.cat((temp, xNew), dim=2) # (samples, embedDim, 2, 11, 11)
            x = self.convBlock(x) # (samples, embedDim, 1, 11, 11)
            x = x.reshape(x.shape[0], x.shape[1], 11, 11) # (samples, embedDim, 11, 11)
            x = x.reshape(x.shape[0], x.shape[1], -1) # (samples, embedDim, patches)
            x = x.transpose(1, 2) # (samples, patches, embedDim)

        x = self.norm(x) # (samples, patches, embedDim)
        x = x.transpose(1,2)
        x = self.pool(x)[:, :, 0] # (samples, embedDim)
        x = self.head(x)


        return x

In [ ]:
import os
import torch
import torch.utils.data as dataf
import scipy.io as io
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
from sklearn import manifold, datasets
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import squareform
import torch.nn as nn
from sklearn.manifold import TSNE
import auxil
from torchsummary import summary

datasetNames = ["houston", "MUUFL", "botswana"]
testSizeNumber = 100
batchsize = 64
EPOCH = 500
LR = 4.95e-4
HSIOnly = True
FileName = "3dConvSST"


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)


for datasetName in datasetNames:
  print("----------------------------------Training for ",datasetName," ---------------------------------------------")
  try:
      os.makedirs(datasetName)
  except FileExistsError:
      pass

  data1Name = datasetName

  X = io.loadmat('./HSI_Dataset/'+data1Name+'Tr.mat')['HSI_tr']
  Y = io.loadmat('./HSI_Dataset/'+data1Name+'Tr.mat')['Label_tr']
  NC = X.shape[3]
  X_Test = io.loadmat('./HSI_Dataset/'+data1Name+'Te.mat')['HSI_te']
  Y_Test = io.loadmat('./HSI_Dataset/'+data1Name+'Te.mat')['Label_te']


  TrainPatch1 = torch.from_numpy(X).to(torch.float32)
  TrainPatch1 = TrainPatch1.permute(0,3,1,2).to(torch.float32)
  TrainLabel1 = torch.from_numpy(Y)
  TrainLabel1 = TrainLabel1.long()
  TrainLabel1 = TrainLabel1.reshape(-1)

  TestPatch1 = torch.from_numpy(X_Test).to(torch.float32)
  TestPatch1 = TestPatch1.permute(0,3,1,2).to(torch.float32)
  TestLabel1 = torch.from_numpy(Y_Test)
  TestLabel1 = TestLabel1.long()
  TestLabel1 = TestLabel1.reshape(-1)

  Classes = len(np.unique(TrainLabel1))
  dataset = dataf.TensorDataset(TrainPatch1,TrainLabel1)
  if data1Name in ['Berlin']:
      train_loader = dataf.DataLoader(dataset, batch_size=batchsize, shuffle=True, num_workers= 0)
  else:
      train_loader = dataf.DataLoader(dataset, batch_size=batchsize, shuffle=True, num_workers= 4)
  print("HSI Train data shape = ", TrainPatch1.shape)
  print("Train label shape = ", TrainLabel1.shape)

  print("HSI Test data shape = ", TestPatch1.shape)
  print("Test label shape = ", TestLabel1.shape)

  print("Number of Classes = ", Classes)



  KAPPA = []
  OA = []
  AA = []
  ELEMENT_ACC = np.zeros((1, Classes))

  set_seed(42)
  for iterNum in range(1):
      cnn = VisionTransformer(in_chans=NC, n_classes=Classes, embed_dim=64, depth=2, n_heads=8).cuda()
      summary(cnn, (NC, 11, 11), device='cuda')
      optimizer = torch.optim.Adam(cnn.parameters(), lr=LR,weight_decay=5e-3)
      loss_func = nn.CrossEntropyLoss()
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)
      BestAcc = 0
      torch.cuda.synchronize()
      # train and test the designed model
      for epoch in range(EPOCH):
          for step, (b_x1, b_y) in enumerate(train_loader):
              # move train data to GPU
              b_x1 = b_x1.cuda()
              b_y = b_y.cuda()
              out1 = cnn(b_x1)
              loss = loss_func(out1, b_y)


              optimizer.zero_grad()  # clear gradients for this training step
              loss.backward()  # backpropagation, compute gradients
              optimizer.step()  # apply gradients

              if step % 10 == 0:
                  cnn.eval()
                  pred_y = np.empty((len(TestLabel1)), dtype='float32')
                  number = len(TestLabel1) // testSizeNumber
                  for i in range(number):
                      temp = TestPatch1[i * testSizeNumber:(i + 1) * testSizeNumber, :, :]
                      temp = temp.cuda()


                      temp2 = cnn(temp)
                      temp3 = torch.max(temp2, 1)[1].squeeze()
                      pred_y[i * testSizeNumber:(i + 1) * testSizeNumber] = temp3.cpu()
                      del temp, temp2, temp3


                  if (i + 1) * testSizeNumber < len(TestLabel1):
                      temp = TestPatch1[(i + 1) * testSizeNumber:len(TestLabel1), :, :]
                      temp = temp.cuda()
                      temp2 = cnn(temp)
                      temp3 = torch.max(temp2, 1)[1].squeeze()
                      pred_y[(i + 1) * testSizeNumber:len(TestLabel1)] = temp3.cpu()
                      del temp, temp2, temp3

                  pred_y = torch.from_numpy(pred_y).long()
                  accuracy = torch.sum(pred_y == TestLabel1).type(torch.FloatTensor) / TestLabel1.size(0)

                  print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '| test accuracy: %.4f' % (accuracy*100))

                  # save the parameters in network
                  if accuracy > BestAcc:
                      BestAcc = accuracy
                      torch.save(cnn.state_dict(), datasetName+'/weights.pkl')
  #                                   print("Weights = ", w0.cpu().detach().numpy())
  #                                   print(w1.cpu().detach().numpy())


                  cnn.train()
          scheduler.step()
      torch.cuda.synchronize()


      cnn.load_state_dict(torch.load(datasetName+'/weights.pkl'))


      cnn.eval()
      confusion, oa, each_acc, aa, kappa = auxil.reports(TestPatch1, TestLabel1, datasetName, cnn, testSizeNumber)
      print("OA AA, Kappa ACCclass", oa, aa, kappa, each_acc)


  cnn.eval()
  number = len(TestLabel1)//testSizeNumber
  features = []
  # pred_y = []
  labels = pred_y.data.cpu().numpy()
  # print(pred_y)
  # print(len(pred_y), np.min(pred_y), np.max(pred_y))
  for i in range(number):
      temp = TestPatch1[i * testSizeNumber:(i + 1) * testSizeNumber]
      temp = temp.cuda()
      temp2 = cnn(temp)
      fea = temp2
      features.append(fea.cpu().detach().numpy())
      del temp, temp2

  if (i + 1) * testSizeNumber < len(TestLabel1):
      temp = TestPatch1[(i + 1) * testSizeNumber:len(TestLabel1)]
      temp = temp.cuda()
      temp2 = cnn(temp)
      fea = temp2
      features.append(fea.cpu().detach().numpy())
      del temp, temp2


  features = np.concatenate(features)
  print("features = ", features.shape)